In [8]:
using POMDPs, POMDPToolbox
using Distributions
include("POMDPGraph_utils.jl")

rewards (generic function with 1 method)

In [16]:
include("network_library.jl")
g = original_roadnet()
(reward_states,reward_vals) = rewards(g)
mdp = roadnet(0.6,0.9,reward_vals,reward_states,g)

roadnet(0.6, 0.9, [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1000.0], roadnetState[roadnetState(1, false), roadnetState(2, false), roadnetState(3, false), roadnetState(4, false), roadnetState(5, false), roadnetState(6, false), roadnetState(7, false), roadnetState(8, false), roadnetState(9, false), roadnetState(10, false), roadnetState(11, false), roadnetState(12, false), roadnetState(13, true)], {13, 19} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0))

In [17]:
sim(mdp,roadnetState(1),max_steps=1) do s
    println("state is: $s")
    a_set = [:n1,:n2,:n3,:n4]
    a = a_set[rand(1:end)] # choose random action
    println("moving to $a")
    return a
end

state is: roadnetState(1, false)
moving to n3


POMDPToolbox.MDPHistory{roadnetState,Symbol}(roadnetState[roadnetState(1, false), roadnetState(1, false)], Symbol[:n3], [-1.0], 0.9, Nullable{Exception}(), Nullable{Any}())

In [18]:
using DiscreteValueIteration

solver = ValueIterationSolver(max_iterations=1000, belres=1e-3)
policy = ValueIterationPolicy(mdp)

solve(solver, mdp, policy, verbose=true)

[Iteration 1   ] residual:   1E+03 | iteration runtime:      0.859 ms, (  0.000859 s total)
[Iteration 2   ] residual:        900 | iteration runtime:      0.741 ms, (    0.0016 s total)
[Iteration 3   ] residual:        810 | iteration runtime:      0.832 ms, (   0.00243 s total)
[Iteration 4   ] residual:        729 | iteration runtime:      0.579 ms, (   0.00301 s total)
[Iteration 5   ] residual:        656 | iteration runtime:      0.788 ms, (    0.0038 s total)
[Iteration 6   ] residual:        590 | iteration runtime:      0.488 ms, (   0.00429 s total)
[Iteration 7   ] residual:        531 | iteration runtime:      0.537 ms, (   0.00482 s total)
[Iteration 8   ] residual:        478 | iteration runtime:      0.494 ms, (   0.00532 s total)
[Iteration 9   ] residual:        430 | iteration runtime:      0.826 ms, (   0.00614 s total)
[Iteration 10  ] residual:        387 | iteration runtime:      0.530 ms, (   0.00667 s total)
[Iteration 11  ] residual:        349 | iteration run

DiscreteValueIteration.ValueIterationPolicy([513.103 543.471 488.124 488.124; 487.041 468.174 503.743 452.368; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [543.471, 503.743, 491.062, 575.397, 586.094, 703.613, 708.35, 873.358, 589.864, 672.451  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2, 3, 2, 3, 2, 3, 3, 4, 3, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Symbol[:n1, :n2, :n3, :n4], true, roadnet(0.6, 0.9, [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1000.0], roadnetState[roadnetState(1, false), roadnetState(2, false), roadnetState(3, false), roadnetState(4, false), roadnetState(5, false), roadnetState(6, false), roadnetState(7, false), roadnetState(8, false), roadnetState(9, false), roadnetState(10, false), roadnetState(11, false), roadnetState(12, false), roadnetState(13, true)], {13, 19} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)))

In [19]:
for x in vertices(mdp.road_net)
    s = roadnetState(x)
    #display("node $x")
    #display(neighbors(mdp.road_net,s.node)')
    a = action(policy,s)
    display(a)
end

:n2

:n3

:n2

:n3

:n2

...

:n3

:n3

:n3

:n4

:n3

In [20]:
s = roadnetState(4)
hist = HistoryRecorder(max_steps=1000)

hist = simulate(hist,mdp,policy,s)

println("Total discounted reward: $(discounted_reward(hist))")

Total discounted reward: 726.2900000000001


In [14]:
for (s, a, sp) in eachstep(hist, "s,a,sp")
    @printf("s: %-26s  a: %-6s  s': %-26s\n", s, a, sp)
end

s: roadnetState(4, false)      a: n3      s': roadnetState(7, false)    
s: roadnetState(7, false)      a: n3      s': roadnetState(8, false)    
s: roadnetState(8, false)      a: n4      s': roadnetState(13, true)    
